In [1]:
import os

def scan_and_print_files(start_dir):
    # 定义需要检索的后缀
    target_extensions = ('.js', '.jsx')
    
    # 统计找到的文件数量
    count = 0

    print(f"正在扫描目录: {start_dir} ...\n")

    # os.walk 遍历目录
    for root, dirs, files in os.walk(start_dir):
        
        # --- 关键优化: 跳过 node_modules 和 .git 目录 ---
        # 直接修改 dirs 列表，os.walk 会自动忽略被移除的目录
        if 'node_modules' in dirs:
            dirs.remove('node_modules')
        if '.git' in dirs:
            dirs.remove('.git')
        # ---------------------------------------------

        for file in files:
            if file.endswith(target_extensions):
                count += 1
                file_path = os.path.join(root, file)
                
                # 打印分隔符和路径，方便视觉区分
                print("=" * 60)
                print(f"文件路径: {file_path}")
                print("-" * 60)

                try:
                    # 打开并读取文件内容，使用 utf-8 编码
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                        print(content)
                except Exception as e:
                    print(f"[读取错误]: 无法读取文件 {file_path}. 原因: {e}")
                
                print("\n") # 每个文件结束后换行

    print(f"扫描结束。共找到 {count} 个 .js/.jsx 文件。")



target_directory = "src" 

scan_and_print_files(target_directory)

正在扫描目录: src ...

文件路径: src/main.jsx
------------------------------------------------------------
import React from 'react'
import ReactDOM from 'react-dom/client'
import App from './App.jsx'
import './App.css'

ReactDOM.createRoot(document.getElementById('root')).render(
  // StrictMode 在开发环境下会导致 useEffect 执行两次
  // 对于地图初始化来说，如果不处理 destroy 可能会导致双重渲染问题
  // 我们在 App.jsx 里写了 return () => map.destroy()，所以这里开着也是安全的
  <React.StrictMode>
    <App />
  </React.StrictMode>,
)


文件路径: src/App.jsx
------------------------------------------------------------
import React, { useEffect, useRef, useState, useCallback } from 'react';
import './App.css';
import ControlPanel from './components/ControlPanel';
import { extractBuildings } from './utils/buildingCore';
import { extractPOI } from './utils/poiCore';
import { extractBase, mergeGlobalBaseFeatures } from './utils/baseCore';
import { extractRoads } from './utils/roadCore'; // 引入路网提取
import { downloadGeoJSON } from './utils/geoUtils';

// --- 新增：格式